In [87]:
# hide
%reload_ext autoreload
%autoreload 2

In [88]:
from dataFunction import *
from clientClass import *
import diffprivlib.models as dp
import numpy as np
import copy
from make_logreg_data import *
import random
import tensorflow as tf
from FL_utils import *
from HE_functions import *
from typing import Generator
import tenseal as ts
from sklearn.linear_model import LogisticRegression
Interval = tuple[Number, Number]

random.seed(11007303)
np.random.seed(2021)

In [89]:
# Parameters
n_clients = 10
n_features = 10
n_observations = 500


In [90]:
glob_model = LR(n_features)
EPOCHS = 80
poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]

# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 20

# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()


        

In [91]:
# random_overlapping_intervals
# make_features
# make_labels
# KL_divergence
# KL_client
# create_client

# Generate mu, np.random.uniform(total interval)
# 
# cov matrix diagonal, same variance dim
# np.eye(5)*3

# Generate input samples, np.random.multivariate_normal(mu,covmat)
# per client different mu

# KL divergence https://statproofbook.g\ithub.io/P/mvn-kl.html, symmetrisch  (KL(1,2)+KL(2,1))/2
# Mate van overlap binnen alle clients, 
# np.eye(1.5)
# client, multivariate normal distribution uses 1xN vector of mu, and NxN matrix of sigma (cov)


In [92]:
clients_distribution = []
for i in range(n_clients):
    random.seed(11007303)
    np.random.seed(2021)
    mu = np.random.default_rng().uniform(-1,1, n_features)
    rand_cov_num = np.random.default_rng().integers(low=1, high=4)
    cov = np.diag([rand_cov_num]*n_features)
    clients_distribution.append([mu, cov])

In [103]:
np.random.default_rng().integers(low=1, high=4)

1

In [93]:
def KL_divergence(dist1, dist2):
    mu1 = dist1[0]
    cov1 = dist1[1]

    mu2 = dist2[0]
    cov2 = dist2[1]

    mu_dif = mu2 - mu1
    inv_cov2 = np.linalg.inv(cov2)
    trace_cov12 = np.matrix.trace(inv_cov2*cov1)
    det_cov1 = np.linalg.det(cov1)
    det_cov2 = np.linalg.det(cov2)

    return 1/2 *( mu_dif.T @ inv_cov2 @ mu_dif+trace_cov12-np.log(det_cov1/det_cov2)-len(mu1))

In [94]:
kl = np.empty((n_clients, n_clients))
kl_sym = np.empty((n_clients, n_clients))
for i in range(n_clients):
    for j in range(n_clients):
        kl[i,j] = KL_divergence(clients_distribution[i], clients_distribution[j])
        kl_sym[i,j] = (KL_divergence(clients_distribution[i], clients_distribution[j]) +KL_divergence(clients_distribution[j], clients_distribution[i]))/2


In [95]:
pd.DataFrame(kl)

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.896710,2.278748,2.064220,0.673314,1.629188,1.144402,6.363325,1.467728,1.162355
1,0.896710,0.000000,1.981403,2.016923,0.582119,1.251368,1.191558,7.254065,1.995905,2.369730
2,1.564708,1.366478,0.000000,1.403626,1.443618,1.554487,1.522585,5.032873,1.471273,2.287673
3,1.421689,1.390158,1.403626,0.000000,1.117339,1.202224,0.879221,6.183932,2.009755,0.848504
4,0.673314,0.582119,2.097113,1.607694,0.000000,0.718190,0.707649,7.554651,1.593157,1.757193
5,1.629188,1.251368,2.263417,1.735023,0.718190,0.000000,1.035190,8.952933,2.212907,2.278684
6,1.144402,1.191558,2.215564,1.250517,0.707649,1.035190,0.000000,9.011290,2.534408,1.775804
7,2.778524,3.075437,2.715040,3.290570,3.175632,3.641726,3.661179,0.000000,1.372483,2.552613
8,1.024028,1.376146,1.471273,2.009755,1.107647,1.520814,1.735148,2.347758,0.000000,1.317049
9,0.820446,1.625363,2.287673,0.848504,1.217005,1.564665,1.229412,4.708019,1.317049,0.000000


In [96]:
pd.DataFrame(kl_sym)

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.896710,1.921728,1.742954,0.673314,1.629188,1.144402,4.570924,1.245878,0.991400
1,0.896710,0.000000,1.673940,1.703540,0.582119,1.251368,1.191558,5.164751,1.686025,1.997546
2,1.921728,1.673940,0.000000,1.403626,1.770366,1.908952,1.869074,3.873957,1.471273,2.287673
3,1.742954,1.703540,1.403626,0.000000,1.362516,1.468623,1.064869,4.737251,2.009755,0.848504
4,0.673314,0.582119,1.770366,1.362516,0.000000,0.718190,0.707649,5.365141,1.350402,1.487099
5,1.629188,1.251368,1.908952,1.468623,0.718190,0.000000,1.035190,6.297330,1.866860,1.921674
6,1.144402,1.191558,1.869074,1.064869,0.707649,1.035190,0.000000,6.336234,2.134778,1.502608
7,4.570924,5.164751,3.873957,4.737251,5.365141,6.297330,6.336234,0.000000,1.860121,3.630316
8,1.245878,1.686025,1.471273,2.009755,1.350402,1.866860,2.134778,1.860121,0.000000,1.317049
9,0.991400,1.997546,2.287673,0.848504,1.487099,1.921674,1.502608,3.630316,1.317049,0.000000


In [97]:
def make_labels(X: np.ndarray, thresholds: list[Number]) -> np.ndarray:
    thresholds = np.array(thresholds)[:, None]
    return (X @ thresholds) > 0


In [98]:
clients = []
for i in clients_distribution:
    samples = np.random.default_rng().multivariate_normal(i[0], i[1], n_observations)
    x= pd.DataFrame(samples)
    # clients_distribution.append([mu, cov])
    weights = np.random.default_rng().integers(low=-10, high=10, size=n_features)
    y = make_labels(x, weights ).replace({True: 1, False: 0})
    # print(type(y.astype(int)))
    lr = 3
    lr_decay = 1
    client_model = copy.deepcopy(glob_model)
    clients.append(Client(i, x, y, client_model, lr, torch.nn.BCELoss()))

In [99]:
validation_X_set = torch.tensor(())
validation_y_set = torch.tensor(())
for i in range(len(clients)):
    validation_X_set = torch.cat((validation_X_set, clients[i].X_test), 0)
    validation_y_set = torch.cat((validation_y_set, clients[i].y_test), 0)

In [100]:
def FL_proces(clients, validation_X_set, validation_y_set):
    loss_train = []
    net_best = None
    best_acc = None
    best_epoch = None
    results = []
    min_loss_client = []
    glob_model = LR(n_features)
    glob_model.eval()
    enrypted_state_dicts= None
    for iter in range(50):
        loss_locals = []
        client_state_dicts = []
        for client in clients:
            client.set_state_dict(glob_model.state_dict())
            client_state_dict, loss = client.train()
            
            loss_locals.append(copy.deepcopy(loss))
            min_loss_client.append(min(loss))
            client_state_dicts.append(client_state_dict)

        enrypted_state_dicts = encrypt_state_dicts(copy.deepcopy(client_state_dicts), ctx_eval)
        averaged_encrypted_state_dict = average_state_dict(enrypted_state_dicts)
        decrypted_state_dicts = decrypt_state_dicts(averaged_encrypted_state_dict)
        glob_model.load_state_dict(decrypted_state_dicts)

        loss_avg = sum(min_loss_client) / len(min_loss_client)
        loss_train.append(loss_avg)        
            
        acc_test, loss_test =  accuracy_loss_LR(glob_model,validation_X_set, validation_y_set)

        print('Round {:3d}, Average loss {:.3f}, Test loss {:.3f}, Test accuracy: {:.2f}'.format(
            iter, loss_avg, loss_test, acc_test))


        if best_acc is None or acc_test > best_acc:
            net_best = copy.deepcopy(glob_model)
            best_acc = acc_test
            best_epoch = iter

        results.append(np.array([iter, loss_avg, loss_test, acc_test, best_acc]))
        final_results = np.array(results)
        final_results = pd.DataFrame(final_results, columns=['epoch', 'loss_avg', 'loss_test', 'acc_test', 'best_acc'])

    print('Best model, iter: {}, acc: {}'.format(best_epoch, best_acc))    
    return best_epoch, best_acc

Round   0, Average loss 0.110, Test loss 0.744, Test accuracy: 0.66
Round   1, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round   2, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round   3, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round   4, Average loss 0.110, Test loss 0.749, Test accuracy: 0.66
Round   5, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round   6, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round   7, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round   8, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round   9, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round  10, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round  11, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round  12, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round  13, Average loss 0.110, Test loss 0.750, Test accuracy: 0.66
Round  14, Average loss 0.110, Test loss 0.750, 